In [ ]:
TESTLIB_PATH = "/Users/kai/Projects/AlLib/target/testlib.dylib"    # Set this to the location of the shared object
# TESTLIB_PATH = "/usr/local/TestLib/target/testlib.so"
HOSTNAME = "localhost"                                             # Set this to the name of the host and the port 
PORT = 24960                                                       # number that the Alchemist driver is running on

In [ ]:
from alchemist import *
import numpy as np

als = AlchemistSession(worker_buffer_length=10000000)              # Start Alchemist session

als.connect_to_alchemist(HOSTNAME, PORT)

In [ ]:
als.request_workers(2) 

In [ ]:
TestLib = als.load_library("TestLib", TESTLIB_PATH)

In [ ]:
A = np.random.rand(2000,1000)

A_handle = als.send_matrix(matrix=A, print_times=True, layout="VC_STAR")

In [ ]:
AA = als.fetch_matrix(A_handle, print_times=True)

In [ ]:
import matplotlib.pyplot as plt

ax = plt.imshow(A[1900:2000, :] - AA[1900:2000, :])
plt.colorbar(ax)
plt.show()

print(np.linalg.norm(A - AA))

In [ ]:
in_byte = 9
in_char = 'y'
in_short = 9876
in_int = 987654321
in_long = 98765432123456789
in_float = 77.77777777
in_double = 88.88888888888888888
in_string = "test string"

TestLib.greet(in_byte, in_char, in_short, in_int, in_long, in_float, in_double, in_string)

In [ ]:
rank = 20
S_handle, U_handle, V_handle = TestLib.truncated_svd(A_handle, rank)

In [ ]:
S = als.fetch_matrix(S_handle, print_times=True)

print(np.flipud(S).transpose())

In [ ]:
V = als.fetch_matrix(V_handle, print_times=True)
U = als.fetch_matrix(U_handle, print_times=True)
S = als.fetch_matrix(S_handle, print_times=True)

In [ ]:
u, s, vh = np.linalg.svd(A, full_matrices=True)

In [ ]:
print(np.linalg.norm(abs(V.T) - abs(np.flipud(vh[0:rank,:]))))
print(np.linalg.norm(abs(U) - abs(np.fliplr(u[:,0:rank]))))
print(np.linalg.norm(np.flip(S).T - s[0:rank]))

In [ ]:
als.stop()